In [10]:
desc_folder = './results'
colName_mid = 'Compound Name'

desc_rdkit = True
desc_fps = True
desc_cx = True
desc_custom = True
model_type = 'regression'    # 'classification'

##
descType_list = []
if desc_custom:
    descType_list.append('custom')
if desc_rdkit:
    descType_list.append('rdkit')
if desc_fps:
    descType_list.append('fingerprints')
if desc_cx:
    descType_list.append('chemaxon')
print(f"\tSelected descriptors types: {descType_list}")

	Selected descriptors types: ['custom', 'rdkit', 'fingerprints', 'chemaxon']


In [11]:
import pandas as pd

## load the expt outcome
dataTable_y = pd.read_csv(f'{desc_folder}/outcome_expt.csv')
print(dataTable_y.shape)
dataTable_y.head(3)

(38, 2)


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num)
0,KT-0001674,0.298013
1,KT-0001675,0.767547
2,KT-0001676,0.186651


In [20]:
mydict = {'a': 1, 'b':2}
a = mydict['a']
b = mydict['a']
a = 2
a, b

(2, 1)

In [18]:
a = 2
mydict

{'a': 1, 'b': 2}

In [12]:
## load all descriptor tables and merge together
dataTable_merged = dataTable_y

for descType in descType_list:
    dataTable = pd.read_csv(f'{desc_folder}/descriptors_{descType}{norm}.csv') 
    print(f"\tThere are total <{dataTable.shape[1]-1}> {descType} descriptors for <{dataTable.shape[0]}> molecules")
    dataTable_merged = dataTable_merged.merge(right=dataTable, on='Compound Name', how='left')
print(f"\tThe merged data table has <{dataTable_merged.shape[0]}> molecules and <{dataTable_merged.shape[1]-1}> descriptors")
dataTable_merged.head(5)

	There are total <2> custom descriptors for <38> molecules
	There are total <140> rdkit descriptors for <38> molecules
	There are total <2048> fingerprints descriptors for <38> molecules
	There are total <33> chemaxon descriptors for <38> molecules
	The merged data table has <38> molecules and <2224> descriptors


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num),custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,KT-0001674,0.298013,-0.012508,-0.449259,1.820147,1.820147,0.753577,1.248948,0.347800,-0.560154,...,-0.766791,0.171082,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
1,KT-0001675,0.767547,0.034798,1.291708,1.808079,1.808079,2.133891,1.271090,0.299537,-0.447405,...,-0.766791,0.737510,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.178726
2,KT-0001676,0.186651,-2.419224,-0.765732,-1.339989,-1.339989,-0.372173,-0.569409,-0.658017,-1.698210,...,-1.386750,-0.142262,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689
3,KT-0001685,0.482284,0.250439,-0.514664,1.794744,1.794744,0.092685,1.236181,0.308173,-0.560154,...,-0.766791,-0.129833,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
4,KT-0001686,0.642138,-2.476299,-0.667554,-1.376570,-1.376570,-0.139945,-0.572946,-0.646185,-1.698210,...,-1.386750,-0.358406,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689


In [13]:
## get descriptors (X)
X = dataTable_merged.iloc[:, 2:]
y = dataTable_merged.iloc[:, 1:2]
print(X.shape, y.shape)

(38, 2223) (38, 1)


##### Variance-based Filter

In [14]:
def VarianceFilter(Descriptors, threshold=0):
    ## Removing features with no variance
    from sklearn.feature_selection import VarianceThreshold
    variance_filter = VarianceThreshold(threshold=threshold)
    variance_filter.fit_transform(Descriptors)

    ## generate variance table
    desc_list = list(Descriptors.columns)
    variance_list = variance_filter.variances_.tolist()
    variance_dict, desc_sele = {}, []
    for i in range(len(desc_list)):
        variance_dict[i] = {}
        variance_dict[i]['Descriptor'] = desc_list[i]
        variance_dict[i]['Variance'] = variance_list[i]
        if variance_list[i] > threshold:
            variance_dict[i]['Select'] = 'Yes'
            desc_sele.append(desc_list[i])
        else:
            variance_dict[i]['Select'] = 'No'
    variance_table = pd.DataFrame.from_dict(variance_dict).T
    ## print results
    print(f"\tIn total <{len(desc_list)}> desc, there are <{len(desc_sele)}> selected, cutoff is {threshold}.")
    return desc_sele, variance_table

##
desc_sele, variance_table = VarianceFilter(Descriptors=X, threshold=0.0001)
variance_table.head(3)

	In total <2223> desc, there are <937> selected, cutoff is 0.0001.


/mnt/data0/software/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


,Descriptor,Variance,Select
0,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,1.0,Yes
1,custDesc_ADME MDCK(WT) Permeability;Mean;B to ...,1.0,Yes
2,rd_MaxAbsEStateIndex,1.0,Yes


In [16]:
def regularization_based_selection(X, y, model_type='regression'):
    if model_type == 'regression':
        from sklearn import linear_model
        estimator = linear_model.Lasso(alpha=0.1)
    elif model_type == 'classification':
        from sklearn.svm import LinearSVC
        estimator = LinearSVC(C=0.01, penalty="l1", dual=False)

    estimator.fit(X, y)
    ##
    from sklearn.feature_selection import SelectFromModel
    fs_model = SelectFromModel(estimator=estimator, prefit=True)
    return fs_model

fs_model = regularization_based_selection(X=X, y=y, model_type='regression')


ValueError: Input X contains NaN.
Lasso does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values